In [ ]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [ ]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [1]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv('exoplanet_data.csv')
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [3]:
#target value counts
df['koi_disposition'].value_counts()

FALSE POSITIVE    3504
CONFIRMED         1800
CANDIDATE         1687
Name: koi_disposition, dtype: int64

In [4]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_time0bk', 'koi_impact','koi_duration', 'koi_depth', 'koi_prad', 
       'koi_teq', 'koi_insol', 'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff',  'koi_slogg', 
       'koi_srad',  'ra', 'dec','koi_kepmag']]

In [5]:
selected_features.shape

(6991, 20)

# Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
target = df['koi_disposition']

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train,X_test,y_train,y_test = train_test_split(selected_features,target,test_size = 0.2)

In [9]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
1989,0,1,0,0,110.321673,197.365128,0.715,13.36240,7296.6,18.60,761,79.11,2217.1,1,9244,4.159,2.025,290.85458,42.086922,11.375
5423,0,0,0,0,7.188641,133.145660,0.424,2.36400,133.5,1.30,987,223.87,9.3,2,5683,4.326,1.115,291.81290,37.555870,14.595
6862,0,0,1,1,2.178187,132.004770,0.553,2.68600,115.8,1.03,1347,776.45,15.4,1,5584,4.428,0.932,288.04980,46.358330,15.123
4010,0,1,0,0,0.656201,132.099430,1.439,4.17901,174860.0,112.77,2157,5094.25,2065.8,1,6025,4.472,0.999,298.53040,40.398460,15.168
6075,1,0,0,1,1.507101,132.093220,0.336,5.80970,497660.0,131.16,2378,7555.78,711.8,1,6919,4.099,1.748,297.26019,43.306358,14.150


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [10]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(selected_features)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
# Label-endoce data set
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [14]:
# one-hot encoding
from tensorflow.keras.utils import to_categorical
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [16]:
y_train_categorical.shape

(5592, 3)

# Train the Model
## Neural Networks with Keras


In [17]:
# define the model layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential()
# 20 selected features and 3 output classes
model.add(Dense(units=100,activation='relu',input_dim=20))
model.add(Dense(units=3,activation='softmax'))

In [18]:
# Compile the model
model.compile(optmizer = 'adam',loss='categorical_crossentropy',metrics= ['accuracy'])

In [19]:
# Fit the model
model.fit(X_train_scaled,y_train_categorical,epochs = 150,shuffle=True,verbose=2)

Train on 5592 samples
Epoch 1/150
5592/5592 - 1s - loss: 0.7354 - accuracy: 0.6273
Epoch 2/150
5592/5592 - 0s - loss: 0.4173 - accuracy: 0.7858
Epoch 3/150
5592/5592 - 0s - loss: 0.3796 - accuracy: 0.7992
Epoch 4/150
5592/5592 - 0s - loss: 0.3727 - accuracy: 0.7979
Epoch 5/150
5592/5592 - 0s - loss: 0.3670 - accuracy: 0.8026
Epoch 6/150
5592/5592 - 0s - loss: 0.3648 - accuracy: 0.8047
Epoch 7/150
5592/5592 - 0s - loss: 0.3630 - accuracy: 0.8040
Epoch 8/150
5592/5592 - 0s - loss: 0.3615 - accuracy: 0.8094
Epoch 9/150
5592/5592 - 0s - loss: 0.3604 - accuracy: 0.8097
Epoch 10/150
5592/5592 - 0s - loss: 0.3589 - accuracy: 0.8095
Epoch 11/150
5592/5592 - 0s - loss: 0.3578 - accuracy: 0.8131
Epoch 12/150
5592/5592 - 0s - loss: 0.3561 - accuracy: 0.8122
Epoch 13/150
5592/5592 - 0s - loss: 0.3556 - accuracy: 0.8140
Epoch 14/150
5592/5592 - 0s - loss: 0.3547 - accuracy: 0.8190
Epoch 15/150
5592/5592 - 0s - loss: 0.3536 - accuracy: 0.8149
Epoch 16/150
5592/5592 - 0s - loss: 0.3542 - accuracy: 0.

Epoch 133/150
5592/5592 - 0s - loss: 0.2938 - accuracy: 0.8628
Epoch 134/150
5592/5592 - 0s - loss: 0.2943 - accuracy: 0.8668
Epoch 135/150
5592/5592 - 0s - loss: 0.2924 - accuracy: 0.8666
Epoch 136/150
5592/5592 - 0s - loss: 0.2919 - accuracy: 0.8634
Epoch 137/150
5592/5592 - 0s - loss: 0.2918 - accuracy: 0.8653
Epoch 138/150
5592/5592 - 0s - loss: 0.2913 - accuracy: 0.8661
Epoch 139/150
5592/5592 - 0s - loss: 0.2912 - accuracy: 0.8653
Epoch 140/150
5592/5592 - 0s - loss: 0.2888 - accuracy: 0.8684
Epoch 141/150
5592/5592 - 0s - loss: 0.2907 - accuracy: 0.8702
Epoch 142/150
5592/5592 - 0s - loss: 0.2900 - accuracy: 0.8716
Epoch 143/150
5592/5592 - 0s - loss: 0.2888 - accuracy: 0.8714
Epoch 144/150
5592/5592 - 0s - loss: 0.2889 - accuracy: 0.8677
Epoch 145/150
5592/5592 - 0s - loss: 0.2869 - accuracy: 0.8709
Epoch 146/150
5592/5592 - 0s - loss: 0.2868 - accuracy: 0.8668
Epoch 147/150
5592/5592 - 0s - loss: 0.2874 - accuracy: 0.8691
Epoch 148/150
5592/5592 - 0s - loss: 0.2867 - accuracy:

In [21]:
# quantifying the model
# evaluate the model using the testing data
model_loss,model_accuracy = model.evaluate(X_test_scaled,y_test_categorical,verbose=2)
print(f'Loss:{model_loss},Accuracy:{model_accuracy}')

1399/1 - 0s - loss: 0.3509 - accuracy: 0.8649
Loss:0.3192485255842297,Accuracy:0.8649035096168518


In [28]:
# Use the first 15 test data values to make a prediction and compare with true value
raw_predictions = model.predict_classes(X_test_scaled[:10])
predictions = list(label_encoder.inverse_transform(raw_predictions))
pd.DataFrame({'Predicted class':predictions,'Actual labels':list(y_test[:10])},columns = ['Predicted class','Actual labels'])

,Predicted class,Actual labels
0,CONFIRMED,CANDIDATE
1,CANDIDATE,CANDIDATE
2,CANDIDATE,CANDIDATE
3,FALSE POSITIVE,FALSE POSITIVE
4,FALSE POSITIVE,FALSE POSITIVE
5,CONFIRMED,CANDIDATE
6,FALSE POSITIVE,FALSE POSITIVE
7,FALSE POSITIVE,FALSE POSITIVE
8,FALSE POSITIVE,FALSE POSITIVE
9,FALSE POSITIVE,FALSE POSITIVE
